# MODEL 2 (Keras) - 1lakh data

## Using the following features:
    
1) CommOrRes  
2) JobCount  
3) ClaimCount  
4) NoteCount  
5) PhotoCount  
6) PolicyHolderType  
7) Display Name  
8) Division Name  
9) Month

In [1]:
import pandas as pd
import numpy as np

cleaned = pd.read_csv('datasets/cleaned_Izends_Data_Thru201712_ver5.csv',index_col=0)

cleaned['EstimateTotal_bins'] = pd.qcut(cleaned['EstimateTotal'],10,labels = ["a", "b", "c", "d","e","f","g","h","i","j"])

/home/ambuj/izenda_fullcycle/environment/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
cleaned['EstimateTotal_bins'] = cleaned['EstimateTotal_bins'].astype(str)

In [10]:
# To see distribution in the sample
# cleaned_sample = cleaned.sample(100000,random_state=42)
# cleaned_sample.EstimateTotal_bins.value_counts()

In [3]:
# Model 2

cleaned_X_mod2 = cleaned[['DisplayName', 'DivisionName','LossMo','CommOrRes','PolicyHolderType','NoteCount','PhotoCount','JobCount','ClaimCount','LossYearMo']]
cleaned_X_mod2_dummies = pd.get_dummies(cleaned_X_mod2,sparse=True)

In [4]:
cleaned_X_mod2.head()

,DisplayName,DivisionName,LossMo,CommOrRes,PolicyHolderType,NoteCount,PhotoCount,JobCount,ClaimCount,LossYearMo
2,Consulting,Inventory,12,Residential,Individual,1.0,0,1,1,201712
12,Contents,Lyons Textile Restoration,5,Residential,Individual,20.0,99,1,1,201405
20,Contents,Lyons Textile Restoration,10,Residential,Individual,7.0,6,1,1,201410
21,Contents,Lyons Textile Restoration,10,Residential,Individual,18.0,81,1,1,201410
25,Contents,Lyons Textile Restoration,12,Residential,Individual,25.0,109,1,1,201412


In [5]:
cleaned_X_mod2_dummies.head()

,LossMo,NoteCount,PhotoCount,JobCount,ClaimCount,LossYearMo,DisplayName_BioHazard,DisplayName_Carpet Cleaning,DisplayName_Consulting,DisplayName_Contents,...,DivisionName_xDuct,DivisionName_xFire,DivisionName_xMold Remediation,DivisionName_xWater Mitigation,DivisionName_Évaluation et inspection,CommOrRes_Commercial,CommOrRes_Other,CommOrRes_Residential,PolicyHolderType_Company,PolicyHolderType_Individual
2,12,1.0,0,1,1,201712,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
12,5,20.0,99,1,1,201405,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
20,10,7.0,6,1,1,201410,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
21,10,18.0,81,1,1,201410,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
25,12,25.0,109,1,1,201412,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [6]:
cleaned_y_mod2 = cleaned['EstimateTotal_bins']
cleaned_X_mod2_dummies['EstimateTotal_bins'] = cleaned_y_mod2

In [7]:
test2017_mod2=cleaned_X_mod2_dummies[cleaned_X_mod2_dummies["LossYearMo"].apply(lambda x:x>=201701)]
learning_mod2=cleaned_X_mod2_dummies[cleaned_X_mod2_dummies["LossYearMo"].apply(lambda x:x<201701)]

In [9]:
learning_sample = learning_mod2.sample(100000,random_state=42)

In [10]:
X_mod2 = learning_sample[learning_sample.columns.difference(['LossYearMo','EstimateTotal_bins'])]
y_bins_mod2 = learning_sample['EstimateTotal_bins']
y_bins_mod2_dummies = pd.get_dummies(y_bins_mod2,sparse=True)

from sklearn.model_selection import train_test_split
X_train_mod2, X_val_mod2, y_train_mod2, y_val_mod2 = train_test_split(X_mod2, y_bins_mod2_dummies, test_size=0.33, random_state=42)

In [11]:
test2017_X_mod2 = test2017_mod2[test2017_mod2.columns.difference(['LossYearMo','EstimateTotal','EstimateTotal_bins'])]
test2017_y_mod2 = test2017_mod2['EstimateTotal_bins']
test2017_y_mod2_dummies = pd.get_dummies(test2017_y_mod2,sparse=True)

In [12]:
# Modelling
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 

Using TensorFlow backend.


In [13]:
model = Sequential()
model.add(Dense(512, input_shape=(X_train_mod2.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
# if(best_params['num_layers'] == 'two_hidden'):
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train_mod2.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='rmsprop')

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = History()

In [34]:
model.fit(X_train_mod2, y_train_mod2.values,
          batch_size=128,
          epochs=500,
          callbacks=[early_stop, history],
          verbose=2,
          validation_data=(X_val_mod2,y_val_mod2.values))

# # Calculating accuracy on 2017 test data
# loss,acc = model.evaluate(test2017_X_mod2,test2017_y_mod2_dummies,verbose=0)
# print(acc)

Train on 67000 samples, validate on 33000 samples
Epoch 1/500
 - 4s - loss: 2.0887 - acc: 0.2093 - val_loss: 2.1006 - val_acc: 0.2062
Epoch 2/500
 - 4s - loss: 2.0844 - acc: 0.2114 - val_loss: 2.0934 - val_acc: 0.2094
Epoch 3/500
 - 4s - loss: 2.0800 - acc: 0.2129 - val_loss: 2.0858 - val_acc: 0.2105
Epoch 4/500
 - 4s - loss: 2.0766 - acc: 0.2144 - val_loss: 2.0923 - val_acc: 0.2035
Epoch 5/500
 - 4s - loss: 2.0755 - acc: 0.2158 - val_loss: 2.0936 - val_acc: 0.2068
Epoch 6/500
 - 4s - loss: 2.0718 - acc: 0.2179 - val_loss: 2.0958 - val_acc: 0.2069
Epoch 7/500
 - 4s - loss: 2.0690 - acc: 0.2174 - val_loss: 2.0887 - val_acc: 0.2038
Epoch 8/500
 - 4s - loss: 2.0676 - acc: 0.2196 - val_loss: 2.0889 - val_acc: 0.2107
Epoch 9/500
 - 4s - loss: 2.0690 - acc: 0.2187 - val_loss: 2.0849 - val_acc: 0.2098
Epoch 10/500
 - 4s - loss: 2.0639 - acc: 0.2221 - val_loss: 2.0908 - val_acc: 0.2098
Epoch 11/500
 - 4s - loss: 2.0636 - acc: 0.2227 - val_loss: 2.0955 - val_acc: 0.2085
Epoch 12/500
 - 4s - los

In [35]:
test2017_preds_mod2 = model.predict_classes(test2017_X_mod2)

In [36]:
mapping_dict = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j'}

In [38]:
test2017_preds_mod2_names = pd.Series(test2017_preds_mod2).map(mapping_dict)

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(test2017_y_mod2,test2017_preds_mod2_names)

0.21309368583546903